In [0]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Install TensorFlow
try:
  %tensorflow_version 2.x
except Exception:
  pass

# TensorFlow ≥2.0-preview is required
import tensorflow as tf
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "ann"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [0]:
from tensorflow import keras

In [0]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full,y_train_full),(X_test, y_test) = fashion_mnist.load_data()

In [0]:
X_valid, X_train = X_train_full[:5000]/255., X_train_full[5000:]/255.
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test/255.

In [0]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28,28]))
model.add(keras.layers.Dense(300, activation="relu"))
model.add(keras.layers.Dense(300, activation="relu"))
model.add(keras.layers.Dense(300, activation="relu"))
model.add(keras.layers.Dense(300, activation="relu"))
model.add(keras.layers.Dense(10, activation="softmax"))

In [0]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer='sgd',
              metrics=["accuracy"])

In [22]:
history = model.fit(X_train, y_train, epochs = 5,
                    validation_data=(X_valid,y_valid))

Train on 55000 samples, validate on 5000 samples
Epoch 1/5
55000/55000 [==============================] - 5s 95us/sample - loss: 0.7566 - accuracy: 0.7415 - val_loss: 0.5454 - val_accuracy: 0.8146
Epoch 2/5
55000/55000 [==============================] - 5s 90us/sample - loss: 0.4756 - accuracy: 0.8319 - val_loss: 0.5030 - val_accuracy: 0.8080
Epoch 3/5
55000/55000 [==============================] - 5s 90us/sample - loss: 0.4224 - accuracy: 0.8508 - val_loss: 0.4081 - val_accuracy: 0.8574
Epoch 4/5
55000/55000 [==============================] - 5s 90us/sample - loss: 0.3916 - accuracy: 0.8594 - val_loss: 0.3762 - val_accuracy: 0.8672
Epoch 5/5
55000/55000 [==============================] - 5s 90us/sample - loss: 0.3679 - accuracy: 0.8667 - val_loss: 0.3553 - val_accuracy: 0.8726


In [23]:
model.evaluate(X_test, y_test)

10000/10000 [==============================] - 1s 68us/sample - loss: 0.3890 - accuracy: 0.8603


[0.3890264170408249, 0.8603]

# 모델 저장/CallBack

In [25]:
model.save("my_keras_model.h5")
model = keras.models.load_model("my_keras_model.h5")
model.evaluate(X_test, y_test)

10000/10000 [==============================] - 1s 70us/sample - loss: 0.3890 - accuracy: 0.8603


[0.3890264170408249, 0.8603]

In [0]:
keras.backend.clear_session()

In [0]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5", save_best_only=True)

In [33]:
history = model.fit(X_train, y_train, epochs = 5,
                    validation_data=(X_valid,y_valid),
                    callbacks = [checkpoint_cb])


Train on 55000 samples, validate on 5000 samples
Epoch 1/5
55000/55000 [==============================] - 5s 88us/sample - loss: 0.3136 - accuracy: 0.8843 - val_loss: 0.3451 - val_accuracy: 0.8734
Epoch 2/5
55000/55000 [==============================] - 5s 89us/sample - loss: 0.3036 - accuracy: 0.8885 - val_loss: 0.3377 - val_accuracy: 0.8730
Epoch 3/5
55000/55000 [==============================] - 5s 88us/sample - loss: 0.2924 - accuracy: 0.8927 - val_loss: 0.3207 - val_accuracy: 0.8810
Epoch 4/5
55000/55000 [==============================] - 5s 90us/sample - loss: 0.2851 - accuracy: 0.8955 - val_loss: 0.3153 - val_accuracy: 0.8844
Epoch 5/5
55000/55000 [==============================] - 5s 89us/sample - loss: 0.2759 - accuracy: 0.8993 - val_loss: 0.3434 - val_accuracy: 0.8726


In [34]:
model1 = keras.models.load_model("my_keras_model.h5")
model1.evaluate(X_valid,y_valid)
model.evaluate(X_valid,y_valid)

5000/5000 [==============================] - 0s 65us/sample - loss: 0.3434 - accuracy: 0.8726


[0.34338552367687225, 0.8726]

In [35]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=3,restore_best_weights=True)

history = model.fit(X_train, y_train, epochs = 10,
                    validation_data=(X_valid,y_valid),
                    callbacks = [early_stopping_cb])


Train on 55000 samples, validate on 5000 samples
Epoch 1/10
55000/55000 [==============================] - 5s 90us/sample - loss: 0.2696 - accuracy: 0.9003 - val_loss: 0.3285 - val_accuracy: 0.8788
Epoch 2/10
55000/55000 [==============================] - 5s 89us/sample - loss: 0.2604 - accuracy: 0.9037 - val_loss: 0.3212 - val_accuracy: 0.8846
Epoch 3/10
55000/55000 [==============================] - 5s 88us/sample - loss: 0.2543 - accuracy: 0.9055 - val_loss: 0.2966 - val_accuracy: 0.8922
Epoch 4/10
55000/55000 [==============================] - 5s 87us/sample - loss: 0.2479 - accuracy: 0.9084 - val_loss: 0.3218 - val_accuracy: 0.8840
Epoch 5/10
55000/55000 [==============================] - 5s 88us/sample - loss: 0.2407 - accuracy: 0.9112 - val_loss: 0.3071 - val_accuracy: 0.8916
Epoch 6/10
55000/55000 [==============================] - 5s 88us/sample - loss: 0.2346 - accuracy: 0.9135 - val_loss: 0.3016 - val_accuracy: 0.8902


# 하이퍼파라미터 튜닝

In [0]:
np.random.seed(42)
tf.random.set_seed(42)

In [0]:
def build_model(n_hidden = 10):
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=[28,28]))
    model.add(keras.layers.Dense(n_hidden, activation='relu'))
    model.add(keras.layers.Dense(10,activation='softmax'))
    model.compile(loss="sparse_categorical_crossentropy",
              optimizer='sgd',
              metrics=["accuracy"])
    return model

In [0]:
keras_cls = keras.wrappers.scikit_learn.KerasClassifier(build_model)

In [51]:
keras_cls.fit(X_train, y_train, epochs=5)

Train on 55000 samples
Epoch 1/5
55000/55000 [==============================] - 4s 76us/sample - loss: 0.9789 - accuracy: 0.6648
Epoch 2/5
55000/55000 [==============================] - 4s 73us/sample - loss: 0.5784 - accuracy: 0.8011
Epoch 3/5
55000/55000 [==============================] - 4s 73us/sample - loss: 0.5173 - accuracy: 0.8211
Epoch 4/5
55000/55000 [==============================] - 4s 72us/sample - loss: 0.4878 - accuracy: 0.8294
Epoch 5/5
55000/55000 [==============================] - 4s 73us/sample - loss: 0.4698 - accuracy: 0.8354


In [54]:
from sklearn.model_selection import RandomizedSearchCV

param_distribs={"n_hidden": [10,20,30]}
rnd_search_cv = RandomizedSearchCV(keras_cls, param_distribs, cv=5)
rnd_search_cv.fit(X_train, y_train, epochs=1)


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 3 is smaller than n_iter=10. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


Train on 44000 samples
11000/11000 [==============================] - 1s 65us/sample - loss: 0.6787 - accuracy: 0.7686
Train on 44000 samples
11000/11000 [==============================] - 1s 65us/sample - loss: 0.7071 - accuracy: 0.7505
Train on 44000 samples
11000/11000 [==============================] - 1s 66us/sample - loss: 0.6603 - accuracy: 0.7756
Train on 44000 samples
11000/11000 [==============================] - 1s 68us/sample - loss: 0.6786 - accuracy: 0.7696
Train on 44000 samples
11000/11000 [==============================] - 1s 65us/sample - loss: 0.6976 - accuracy: 0.7680
Train on 44000 samples
11000/11000 [==============================] - 1s 83us/sample - loss: 0.6411 - accuracy: 0.7881
Train on 44000 samples
11000/11000 [==============================] - 1s 64us/sample - loss: 0.6570 - accuracy: 0.7812
Train on 44000 samples
11000/11000 [==============================] - 1s 66us/sample - loss: 0.6412 - accuracy: 0.7879
Train on 44000 samples
11000/11000 [============

RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f38305fa128>,
                   iid='deprecated', n_iter=10, n_jobs=None,
                   param_distributions={'n_hidden': [10, 20, 30]},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring=None, verbose=0)

In [55]:
rnd_search_cv.best_params_

{'n_hidden': 30}

In [56]:
rnd_search_cv.best_score_

0.7874363541603089

In [57]:
rnd_search_cv.best_estimator_

In [58]:
rnd_search_cv.score(X_test,y_test)

10000/10000 [==============================] - 1s 69us/sample - loss: 0.6157 - accuracy: 0.7894


0.7894

In [59]:
model = rnd_search_cv.best_estimator_.model

In [0]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28,28]))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10,activation='softmax'))
model.compile(loss="sparse_categorical_crossentropy",
            optimizer=keras.optimizers.SGD(momentum=0.9),
            metrics=["accuracy"])

In [0]:
optimizer=keras.optimizers.Adam()

# 여러 optimizer 모두 간단히 사용가능하다.

In [0]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28,28]))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10,activation='softmax'))
model.compile(loss="sparse_categorical_crossentropy",
            optimizer=keras.optimizers.SGD(momentum=0.9),
            metrics=["accuracy"])

In [64]:
[name for name in dir(keras.initializers) if not name.startswith("_")]

['Constant',
 'GlorotNormal',
 'GlorotUniform',
 'Identity',
 'Initializer',
 'Ones',
 'Orthogonal',
 'RandomNormal',
 'RandomUniform',
 'TruncatedNormal',
 'VarianceScaling',
 'Zeros',
 'constant',
 'deserialize',
 'get',
 'glorot_normal',
 'glorot_uniform',
 'he_normal',
 'he_uniform',
 'identity',
 'lecun_normal',
 'lecun_uniform',
 'ones',
 'orthogonal',
 'serialize',
 'zeros']

In [0]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28,28]))
model.add(keras.layers.Dense(100, activation='relu', kernel_initializer="he_normal")) # initializer
model.add(keras.layers.Dense(10, activation='softmax'))
model.compile(loss="sparse_categorical_crossentropy",
            optimizer=keras.optimizers.SGD(momentum=0.9),
            metrics=["accuracy"])